<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Sentiment Analysis of Movie Reviews with Spacy and VADER

_Authors: Kiefer Katovich (SF)_

---

### Learning Objectives
- Understand the goal of basic sentiment analysis.
- Calculate sentiment scores manually using a reviews dataset and scores tagged by word.
- Practice using the spacy parser to get out part of speech tags from text.
- Fit a model using sentiment and grammar features.
- Use the VADER sentiment analyzer to get out more accurate sentiment scores and compare the models.

### Lesson Guide
- [Introduction to sentiment analysis](#intro)
- [Load the word sentiment dataset](#load-sen)
    - [Engineer objectivity and positive difference scores](#adj-scores)
    - [Put scores in a part of speech dictionary](#pos-dict)
- [Load the rotten tomatoes review dataset](#rt-reviews)
    - [Restrict reviews to valid lengths and ratings](#subset)
- [Import spacy](#spacy)
    - [Parse all the quotes using spacy's multithreaded parser](#multi)
- [Part of speech features](#pos-features)
- [Assign sentiment scores](#assign)
- [Print out the most positive and most negative reviews](#print-most)
- [Print out the most objective and most subjective reviews](#print-most-obj)
- [Build a model to classify fresh vs. rotten with the sentiment and grammar features](#model)
- [User the VADER library to get better sentiment scores](#vader)
    - [Build a model using the VADER sentiment features](#vader-model)

<a id='intro'></a>

## Introduction to sentiment analysis
---

Sentiment analysis is one of the most popular topics in NLP. Most commonly it is the quantification of text into valence and subjectivity scores.

First we will load in a dataset of pre-coded sentiment scores for positivity and negativity on words. These words are also tagged with their part of speech in the sentence. We can use these valence scores to evaluate the sentiment of rottentomatoes movie reviews. Many packages such as TextBlob come pre-packaged with sentiment scores for words after parsing text, but doing the sentiment parsing manually will show you how it can be done without any "magic".

We will also explore a more advanced sentiment analysis library in python: [VADER](https://github.com/cjhutto/vaderSentiment). We can parse the sentiment of the movie reviews using this package and compare it to our more basic method.



<a id='load-sen'></a>

## Load the word sentiment dataset
---

Below we will load in some pre-tagged positive and negative valence scores for a dictionary of words. Each row of the dataset contains the part of speech, the word, the positive score, and the negative score for the word. A word may appear more than once if it can appear with different part of speech tags. 

These scores are designed so that we can also derive the *objectivity score* of the word from the positive and negative scores.

Objectivity is calculated: 

    1. - (positive_score + negative_score)

Thus if a score has zero positive score and negative score it is completely objective. If a score has, for example, 0.5 positive and 0.5 negative, it may not be any more positive than negative but we can tell that it is subjective (objectivity = 0.).


In [112]:
import pandas as pd
import numpy as np

In [113]:
sen = pd.read_csv('./datasets/sentiment_words_simple.csv')

In [114]:
# A:

sen.head()

,pos,word,pos_score,neg_score
0,adj,.22-caliber,0.0,0.0
1,adj,.22-calibre,0.0,0.0
2,adj,.22_caliber,0.0,0.0
3,adj,.22_calibre,0.0,0.0
4,adj,.38-caliber,0.0,0.0


**Make the part of speech tags uppercase (this will come in handy later when we use Spacy).**

In [115]:
# A:
sen.pos = [i.upper() for i in sen.pos]
sen.head()

,pos,word,pos_score,neg_score
0,ADJ,.22-caliber,0.0,0.0
1,ADJ,.22-calibre,0.0,0.0
2,ADJ,.22_caliber,0.0,0.0
3,ADJ,.22_calibre,0.0,0.0
4,ADJ,.38-caliber,0.0,0.0


<a id='adj-scores'></a>

### Engineer objectivity and positive difference scores

Since subjective vs. objective is embedded in the positive and negative scores, we should extract this and convert the positive and negative into a relative difference scores.

**Calculate two new scores:**

    objectivity = 1. - (pos_score + neg_score)
    pos_vs_neg = pos_score - neg_score
    

In [116]:
# A:
sen['objectivity'] = 1 - (sen.pos_score - sen.neg_score)
sen['pos_vs_neg'] = sen.pos_score - sen.neg_score

In [117]:
sen.head()

,pos,word,pos_score,neg_score,objectivity,pos_vs_neg
0,ADJ,.22-caliber,0.0,0.0,1.0,0.0
1,ADJ,.22-calibre,0.0,0.0,1.0,0.0
2,ADJ,.22_caliber,0.0,0.0,1.0,0.0
3,ADJ,.22_calibre,0.0,0.0,1.0,0.0
4,ADJ,.38-caliber,0.0,0.0,1.0,0.0


<a id='pos-dict'></a>

### Put scores in a part of speech dictionary

The dictionary format of the data will be much easier to index using our parsing functions later on. Create a dictionary where the keys are the four part of speech tags:

    ADJ
    NOUN
    VERB
    ADV

For each key, store a dictionary that contains all of the words for that part of speech with their objectivity and positive vs. negative scores.

In [118]:
# A:
sen_dict = {'ADJ':{},'NOUN':{},'VERB':{},'ADV':{}}
for i,row in enumerate(sen.itertuples()):
    if (i%10000) == 0:
        print i
    sen_dict[row[1]][row[2]] ={'objectivity': row[5],'pos_vs_neg':row[6]}
    
sen_dict

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000


{'ADJ': {'convolute': {'objectivity': 0.875, 'pos_vs_neg': 0.125},
  'funereal': {'objectivity': 1.5, 'pos_vs_neg': -0.5},
  'unscientific': {'objectivity': 1.625, 'pos_vs_neg': -0.625},
  'unsupportable': {'objectivity': 1.625, 'pos_vs_neg': -0.625},
  'systematic': {'objectivity': 1.25, 'pos_vs_neg': -0.25},
  'cussed': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'overaged': {'objectivity': 0.5, 'pos_vs_neg': 0.5},
  'chthonic': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'semicentenary': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'uttered': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'foul': {'objectivity': 1.375, 'pos_vs_neg': -0.375},
  'narcotic': {'objectivity': 1.0416666666665999,
   'pos_vs_neg': -0.041666666666600009},
  'four': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'clotted': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'large-capitalisation': {'objectivity': 1.0, 'pos_vs_neg': 0.0},
  'ornate': {'objectivity': 0.75, 'pos_vs_neg': 0.25},
  'woody': {'objectivity': 1.0, 'pos_

<a id='rt-reviews'></a>

## Load the rotten tomatoes reviews dataset

---

This dataset has:
    
    critic: critic's name
    fresh: fresh vs. rotten rating
    imdb: code for imdb
    publication: where the review was published
    quote: the review snippet
    review_date: date of review
    rtid: rottentomatoes id
    title: name of movie

In [119]:
rt = pd.read_csv('./datasets/rt_critics.csv')

In [120]:
# A:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story
1,Richard Corliss,fresh,114709.0,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559.0,Toy story
2,David Ansen,fresh,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story
3,Leonard Klady,fresh,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story
4,Jonathan Rosenbaum,fresh,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story


In [121]:
rt.shape

(14072, 8)

<a id='subset'></a>

### Restrict data to reviews with valid ratings and reviews over 10 words long

Also clean up the reviews, making a column with the case and punctuation removed.

In [122]:
# A:
rt = rt[rt.fresh != 'none']
rt.fresh = rt.fresh.map(lambda x: 1 if x == 'fresh' else 0)
rt.reset_index(drop = True, inplace = True)
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story
1,Richard Corliss,1,114709.0,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559.0,Toy story
2,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story
3,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story
4,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story


In [123]:
rt.index.values

array([    0,     1,     2, ..., 14046, 14047, 14048])

In [124]:
rt.shape

(14049, 8)

In [125]:
rt['quote_len'] = rt.quote.map(lambda x: len(x.split()))
rt = rt[rt.quote_len >10]
rt.shape

(11215, 9)

In [126]:
import string
rt['qt'] = rt.quote.map(lambda x: unicode(''.join([y for y in list(x.lower()) if y in string.ascii_lowercase + " -'" ])))
rt.qt = rt.qt.map(lambda x: x.replace('-',' '))

<a id='spacy'></a>

## Import spacy

---

The spacy package is the current gold standard for parsing the grammatical structure of text (aside from neural network architectures). We are going to use it to find the part of speech tags for the review words. 

Once we have parsed the tags with spacy, we can assign objectivity and valence scores by finding the match in our sentiment dataset.

Code for loading spacy:
- pip install spacy
- python -m spacy download en

In [127]:
import spacy
en_nlp = spacy.load('en')

**Parse a single quote:**

In [128]:
# A:
tmp = en_nlp(rt.qt.values[0])

In [129]:
tmp

so ingenious in concept design and execution that you could watch it on a postage stamp sized screen and still be engulfed by its charm

**Print out the part of speech tags for each word in the quote:**

In [130]:
# A:
for token in tmp:
    print token.pos_

ADV
ADJ
ADP
NOUN
NOUN
CCONJ
NOUN
ADJ
PRON
VERB
VERB
PRON
ADP
DET
NOUN
NOUN
VERB
NOUN
CCONJ
ADV
VERB
VERB
ADP
ADJ
NOUN


<a id='multi'></a>
### Parse all the quotes using spacy's multithreaded parser

Parsing a lot of text can take quite awhile. Luckily spacy comes with multithreading functionality to speed up the process considerably. Below is code that will parse the quotes across multiple threads and assign them to a list.

In [131]:
# A:
parsed_quotes =[]
for i, parsed in enumerate(en_nlp.pipe(rt.qt.values, batch_size = 50, n_threads=4)):
    assert parsed.is_parsed
    if(i%1000) == 0:
        print i
    parsed_quotes.append(parsed)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


<a id='pos-features'></a>

## Create features with part of speech proportions

---

With our spacy parsed reviews, we have a lot of feature engineering potential even before we get to sentiment. Something simple we could do is calculate the proportion of words in the quote that have each part of speech tag. We can try using these as predictors in a model later.

**Find all the unique part of speech categories in the reviews.**

In [132]:
# A:
unique_pos = []
for parsed in parsed_quotes:
    unique_pos.extend([t.pos_ for t in parsed])
unique_pos = np.unique(unique_pos)
print unique_pos

[u'ADJ' u'ADP' u'ADV' u'CCONJ' u'DET' u'INTJ' u'NOUN' u'NUM' u'PART'
 u'PRON' u'PROPN' u'PUNCT' u'SPACE' u'SYM' u'VERB' u'X']


**Create the proportion columns for each part of speech.**

In [133]:
# A:
for pos in unique_pos:
    rt[pos+'_prop'] = 0.

**Iterate through the reviews and calculate the proportions of each part of speech tag.**

In [134]:
# A:
rt = rt.reset_index(drop=True)
for i, parsed in enumerate(parsed_quotes):
    if (i % 1000) == 0:
        print i
    parsed_len = len(parsed)
    for pos in unique_pos:
        count = len([x for x in parsed if x.pos_ == pos])
        rt.ix[i, pos+'_prop'] = float(count)/parsed_len

0


/Users/timong/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


<a id='assign'></a>

## Assign sentiment scores
---

We will now use the parsed reviews and the sentiment dataset to assign the average objectivity and positive vs. negative scores.

If a word cannot be found in the dataset we can ignore it. If a review has no words that match something in our dataset, will can assign overall neutral scores of `objectivity = 1` and `pos_vs_neg = 0`.

There are definitely problems with this approach, but for now we can keep it "dumb" and see if things improve when we use the VADER analyzer later.

In [135]:
# A:
def scorer(parsed):
    obj_scores, pvn_scores = [], []
    for token in [t for t in parsed if t.pos_ in ['NOUN','VERB','ADV','ADJ']]:
        try:
            obj_scores.append(sen_dict[token.pos_][str(token)]['objectivity'])
            pvn_scores.append(sen_dict[token.pos_][str(token)]['pos_vs_neg'])
        except:
            pass
    if len(obj_scores) == 0:
        obj_scores = [1.]
    if len(pvn_scores) == 0:
        pvn_scores = [0.]
    return [obj_scores, pvn_scores]


scores = []
for i, parsed in enumerate(parsed_quotes):
    if (i % 1000) == 0:
        print i
    scores.append(scorer(parsed))

rt['objectivity_avg'] = [np.mean(x[0]) for x in scores]
rt['pos_vs_neg_avg'] = [np.mean(x[1]) for x in scores]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [136]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,PART_prop,PRON_prop,PROPN_prop,PUNCT_prop,SPACE_prop,SYM_prop,VERB_prop,X_prop,objectivity_avg,pos_vs_neg_avg
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story,24,so ingenious in concept design and execution t...,...,0.000000,0.080000,0.0,0.0,0.0000,0.0,0.200000,0.0,0.938187,0.061813
1,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story,13,a winning animated feature that has something ...,...,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.230769,0.0,1.000000,0.000000
2,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story,17,the film sports a provocative and appealing st...,...,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.055556,0.0,0.924716,0.075284
3,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story,14,an entertaining computer generated hyperrealis...,...,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.125000,0.0,0.951562,0.048438
4,Michael Booth,1,114709.0,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03,9559.0,Toy story,40,as lion king did before it toy story revived t...,...,0.023256,0.046512,0.0,0.0,0.0000,0.0,0.139535,0.0,0.946932,0.053068


<a id='print-most'></a>
## Print out the most positive and most negative reviews
---

Now that we have the average valence for reviews, try printing out the top 10 most positive and top 10 most negative reviews to visually verify that our approach makes sense.

In [106]:
# A:


<a id='print-most-obj'></a>

## Print out the most objective and most subjective reviews
---

Do the same as above, but now sort by the objectivity. What kind of differences do you notice between these? Does our approach actually appear to capture meaningful subjectivity and objectivity in the reviews?

In [19]:
# A:

<a id='model'></a>

## Build a model to classify fresh vs. rotten with the sentiment and grammar features

---

Let's use the features we've created to construct a Logistic Regression to predict whether a review is fresh vs. rotten. 

Don't forget to check the baseline score, and it's a good practice to standardize your predictors.


In [138]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [137]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,PART_prop,PRON_prop,PROPN_prop,PUNCT_prop,SPACE_prop,SYM_prop,VERB_prop,X_prop,objectivity_avg,pos_vs_neg_avg
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story,24,so ingenious in concept design and execution t...,...,0.000000,0.080000,0.0,0.0,0.0000,0.0,0.200000,0.0,0.938187,0.061813
1,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story,13,a winning animated feature that has something ...,...,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.230769,0.0,1.000000,0.000000
2,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story,17,the film sports a provocative and appealing st...,...,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.055556,0.0,0.924716,0.075284
3,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story,14,an entertaining computer generated hyperrealis...,...,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.125000,0.0,0.951562,0.048438
4,Michael Booth,1,114709.0,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03,9559.0,Toy story,40,as lion king did before it toy story revived t...,...,0.023256,0.046512,0.0,0.0,0.0000,0.0,0.139535,0.0,0.946932,0.053068


In [139]:
X = rt[['objectivity_avg','pos_vs_neg_avg','quote_len']+[x for x in rt.columns if x.endswith('_prop')]]
y = rt.fresh.values

ss = StandardScaler()
Xs = ss.fit_transform(X)

logreg_score = cross_val_score(LogisticRegression(), Xs, y, cv = 10)
print 'model score:{}, baseline score:{}'.format(np.mean(logreg_score), np.mean(rt.fresh))

model score:0.640127113107, baseline score:0.615069103879


<a id='vader'></a>

## Use the VADER library to get better sentiment scores
---

The [VADER](https://github.com/cjhutto/vaderSentiment) package for python is a more advanced way to calculate positivity, negativity, and objectivity in our reviews. The github page describes VADER as:

> VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.

You will likely need to install VADER with pip or conda. Instructions can be found on the github page. Once you have it installed you can load the `SentimentIntensityAnalyzer` and parse text.

**Parse a couple of quotes with the `SentimentIntensityAnalyzer` and print out the dictionary of scores using `analyzer.polarity_scores`:

In [140]:
# pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [141]:
# A:
analyzer = SentimentIntensityAnalyzer()
for sentence in rt.quote.values[0:2]:
    vs = analyzer.polarity_scores(sentence)
    print sentence
    print vs

So ingenious in concept, design and execution that you could watch it on a postage stamp-sized screen and still be engulfed by its charm.
{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'compound': 0.4019}
A winning animated feature that has something for everyone on the age spectrum.
{'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'compound': 0.5267}


You can see that these scores look more legitimate. VADER polarity score dictionaries have 4 elements: `neg`, `pos`, `neu` and `compound`. The compound score is a single metric that represents the "overall" valence.

**Calculate the four scores for each review and save them as features in the dataframe.**

In [109]:
# A:



<a id='vader-model'></a>

### Fit a model using the VADER sentiment features

Does this model perform better? 

In [110]:
# A:


<a id='vader-top'></a>

### Print out the top most negative, positive, neutral, and subjective features by VADER score

In [26]:
# A:

